# music21

In [ ]:
import music21

## Load MIDI

In [ ]:
midi_path = '../resources/99.mid'

In [ ]:
midi = music21.converter.parse(midi_path)

In [ ]:
midi

In [ ]:
midi = midi.chordify()

In [ ]:
midi

In [ ]:
midi.show('text')

## Get chord or note

In [ ]:
music21.instrument.partitionByInstrument(midi)

In [ ]:
chord_elem = [e for e in midi.flat if isinstance(e, music21.chord.Chord)]

In [ ]:
chord_elem

In [ ]:
note_elem = [e for e in midi.flat if isinstance(e, music21.note.Note)]

In [ ]:
note_elem

## Transfer

### Chord

In [ ]:
chord_elem[0]

In [ ]:
chord_elem[0].pitches

In [ ]:
chord_elem[0].pitches[0].nameWithOctave

In [ ]:
'.'.join([p.nameWithOctave for p in chord_elem[0].pitches])

In [ ]:
chord_elem[0].duration.quarterLength

### Note

In [ ]:
note_elem[0]

In [ ]:
note_elem[0].nameWithOctave

In [ ]:
note_elem[0].name

In [ ]:
note_elem[0].duration.quarterLength

In [ ]:
[n for n in note_elem if n.isRest]

# Keras

In [3]:
import numpy as np

### Embedding

In [ ]:
from tensorflow.keras.layers import Embedding

In [ ]:
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [ ]:
data.shape

In [ ]:
max_idx = max([max(d) for d in data])
max_idx

In [ ]:
# kerasのreferenceにあるように、+1しないと以下のエラー
# InvalidArgumentError: indices[0,2] = 3 is not in [0, 3) [Op:ResourceGather]
Embedding(max_idx+1, 2)(data)

## Concatenate

In [ ]:
from tensorflow.keras.layers import Concatenate

In [ ]:
data1 = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [ ]:
data2 = np.array([[[9, 10], [11, 12]], [[13, 14], [15, 16]]])

In [ ]:
data1.shape

In [ ]:
Concatenate()([data1, data2])

## LSTM

In [35]:
from tensorflow.keras.layers import Input, LSTM

In [37]:
from tensorflow.keras.models import Model

In [ ]:
# 例えばtimestepが3、単語等が変換されたベクトルの次元数が2
timestep = 3
word_vec_dim = 2
input = Input(shape=(timestep, word_vec_dim))

In [ ]:
# units=5。hとかcが出力数
lstm = LSTM(5, return_sequences=True)(input)

In [ ]:
model = Model(input, lstm)

In [ ]:
# 例えばデータセットが1つで、単語が3つ、単語の変換後ベクトルが2次元の時
data = np.array([1, 2, 3, 4, 5, 6]).reshape((1, 3, 2))

In [ ]:
result = model.predict(data)

In [ ]:
result

In [ ]:
# return_sequences=Trueなので、各timestep毎のhが出力されている
# Falseならshape=(1, 5)になる
result.shape

## Reshape

In [1]:
from tensorflow.keras.layers import Reshape

In [20]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape((4, 1, 2))
data.shape

(4, 1, 2)

In [21]:
# 次元を一つ減らす
Reshape([-1])(data).shape

TensorShape([4, 2])

## RepeatVector

In [63]:
from tensorflow.keras.layers import RepeatVector

In [26]:
data = np.array([1, 2, 3, 4, 5, 6]).reshape((2, 3))
data

array([[1, 2, 3],
       [4, 5, 6]])

In [27]:
RepeatVector(4)(data)

<tf.Tensor: shape=(2, 4, 3), dtype=int64, numpy=
array([[[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]],

       [[4, 5, 6],
        [4, 5, 6],
        [4, 5, 6],
        [4, 5, 6]]])>

## Permute

In [65]:
from tensorflow.keras.layers import Permute

In [31]:
data = np.array([1, 2, 3, 4, 5, 6]).reshape((1, 2, 3))
data

array([[[1, 2, 3],
        [4, 5, 6]]])

In [32]:
# 指定したdimを入れ替える
Permute([2, 1])(data)

<tf.Tensor: shape=(1, 3, 2), dtype=int64, numpy=
array([[[1, 4],
        [2, 5],
        [3, 6]]])>

## Multiply

In [69]:
from tensorflow.keras.layers import Multiply

In [67]:
data1 = np.array([1, 2, 3, 4]).reshape((2, 2))
data2 = np.array([5, 6, 7, 8]).reshape((2, 2))

In [70]:
Multiply()([data1, data2])

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[ 5, 12],
       [21, 32]])>

## Attention

In [71]:
data = np.array([1, 2, 3, 4, 5, 6]).reshape((1, 3, 2))
data

array([[[1, 2],
        [3, 4],
        [5, 6]]])

In [72]:
input = Input(shape=(3, 2))

In [73]:
lstm = LSTM(4, return_sequences=True)(input)

In [74]:
model = Model(input, lstm)

In [75]:
x = model.predict(data)
x

array([[[ 0.04195475,  0.0094991 , -0.11110172, -0.06483597],
        [ 0.05934902, -0.02767825, -0.12050912, -0.10131744],
        [ 0.06594805, -0.11255478, -0.07469919, -0.09016994]]],
      dtype=float32)

In [76]:
x.shape

(1, 3, 4)

In [77]:
from tensorflow.keras.layers import Dense

In [78]:
result = Dense(1, activation='tanh')(x)

In [79]:
result

<tf.Tensor: shape=(1, 3, 1), dtype=float32, numpy=
array([[[ 0.12823488],
        [ 0.11453976],
        [-0.00553648]]], dtype=float32)>

In [80]:
result = Reshape([-1])(result)
result

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.12823488,  0.11453976, -0.00553648]], dtype=float32)>

In [81]:
result = RepeatVector(4)(result)
result

<tf.Tensor: shape=(1, 4, 3), dtype=float32, numpy=
array([[[ 0.12823488,  0.11453976, -0.00553648],
        [ 0.12823488,  0.11453976, -0.00553648],
        [ 0.12823488,  0.11453976, -0.00553648],
        [ 0.12823488,  0.11453976, -0.00553648]]], dtype=float32)>

In [82]:
result = Permute([2, 1])(result)
result

<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[ 0.12823488,  0.12823488,  0.12823488,  0.12823488],
        [ 0.11453976,  0.11453976,  0.11453976,  0.11453976],
        [-0.00553648, -0.00553648, -0.00553648, -0.00553648]]],
      dtype=float32)>

In [83]:
x.shape

(1, 3, 4)

In [84]:
result.shape

TensorShape([1, 3, 4])

In [85]:
Multiply()([x, result])

<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[ 0.00538006,  0.00121812, -0.01424712, -0.00831423],
        [ 0.00679782, -0.00317026, -0.01380309, -0.01160487],
        [-0.00036512,  0.00062316,  0.00041357,  0.00049922]]],
      dtype=float32)>